## 附加题:完全符合真实工作需求，得分较高者会额外 优先进行内推

- 查看B-spline的介绍。

- 使用 cython 实现对输入多列返回 b-spline basis 的操作。

- 注意:禁止使用函数 recursive call。

- 注意:必须要处理异常情况(例如缺失值，inf 等)。


![](https://wikimedia.org/api/rest_v1/media/math/render/svg/aa454138e7ff46975d7401b90f476dab9d1e4f6d)

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/ddcf0107da7f98bd3ea31345ad84202dde74bf04)

# Original Version

In [1]:
import numpy as np

In [2]:
x = 1.5
d = 5
knots = [1.,2.] 
# Note that the number of knots are not enough for order 5 B-spline, so we have to pad the knot vector, but then we have to deal with the 0-division problems, see below.

In [3]:
def bspline_v1(knots: list, p: int, x: float):
    """
    Evaluate B-spline basis of order `degree` at position `x`, given `knots` as an array of real knot locations, i.e. without endpoints padding,
    using the Cox-de Boor recursion formula defined at https://en.wikipedia.org/wiki/De_Boor%27s_algorithm
    """
    knots = np.array( [knots[0]] * degree  +  knots  +  [knots[-1]] * degree ) # pad the knot vector
    Bs = np.zeros((degree+1, knots.shape[0])) # only the upper triangle will be used
    n = Bs.shape[1]
    # base case
    for j in range(len(knots)-1):
        if knots[j] <= x < knots[j+1]: Bs[0][j] = 1
    # calculate bottom-up (order 1 -> degree)
    for p in range(1, degree+1):
        for i in range(n-p-1):
            term1 = 0.
            if knots[i+p] != knots[i]:
               term1 = (x - knots[i]) * Bs[p-1, i] / (knots[i+p] - knots[i])
            term2 = 0.
            if knots[i+p+1] != knots[i+1]:
                term2 = (knots[i+p+1] - x) * Bs[p-1, i+1] / (knots[i+p+1] - knots[i+1])
            Bs[p, i] = term1 + term2
    return Bs


In [4]:
%%timeit
bspline_v1(knots, d, x)

10000 loops, best of 3: 79.6 µs per loop


In [5]:
res1 = bspline_v1(knots, d, x)

- Try to vectorize

In [19]:
def bspline_v2(knots: list, p: int, x: float): 
    # pad knots vector
    knots = np.array( [knots[0]] * p  +  knots  +  [knots[-1]] * p )
    Bs = np.zeros((p+1, knots.shape[0]))
    # base case
    Bs[0][:-1] = 1.*np.logical_and(knots[:-1] <= x, x < knots[1:])    

    for p_ in range(1, p+1):
        tmp = Bs[p_-1][:-p_]
        numer1 = (x - knots[:-1-p_]) * tmp[:-1]
        numer2 = (knots[p_+1:] - x) * tmp[1:]
        denom1 = (knots[p_:-1] - knots[:-p_-1])
        denom2 = (knots[p_+1:]- knots[1:-p_])
        term1 = np.where(denom1 != 0.0,
                        (numer1 / denom1), 
                         0.0)
        term2 = np.where(denom2 != 0.0,
                        (numer2 / denom2), 
                         0.0)
        tmp = term1 + term2
        Bs[p_][:len(tmp)] = tmp
    return Bs

In [20]:
%%timeit
bspline_v2(knots, d, x)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide


The slowest run took 8.97 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 110 µs per loop


In [13]:
res2 = bspline_v2(knots, d, x)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide


In [14]:
diff = np.linalg.norm(res1 - res2)
print(diff)

0.0


- It actually got worse! The extra copying and sanity-checking is inefficient.

However, since we only need the last order Bs_p, we do not need to save all Bs_0 ~ Bs_p-1.

In [24]:
def bspline_v3(knots: list, p: int, x: float): 
    # pad knots vector
    knots = np.array( [knots[0]] * p  +  knots  +  [knots[-1]] * p )
    # base case
    B = 1.*np.logical_and(knots[:-1] <= x, x < knots[1:])

    for p_ in range(1, p+1):
        numer1 = (x - knots[:-1-p_]) * B[:-1]
        numer2 = (knots[p_+1:] - x) * B[1:]
        denom1 = (knots[p_:-1] - knots[:-p_-1])
        denom2 = (knots[p_+1:]- knots[1:-p_])

        term1 = np.where(denom1 != 0.0,
                        (numer1 / denom1), 
                        0.0)
        term2 = np.where(denom2 != 0.0,
                        (numer2 / denom2), 
                        0.0)
        B = term1 + term2
    return B

In [25]:
%%timeit
bspline_v3(knots, d, x)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide


The slowest run took 4.28 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 101 µs per loop


- Not good still. The zero-division problem cannot be handled well with vectorization.

# Cython Version

In [26]:
%load_ext Cython

In [35]:
%%cython -a
import numpy as np
cimport numpy as np
cimport cython

ctypedef np.float64_t DTYPE_f
ctypedef np.int64_t DTYPE_i
   

# @cython.boundscheck(False) cannot use it in jupyter notebook, crashed the session for unknown reason
# @cython.wraparound(False)
def bspline_v4(ks, int degree, double u): 

    cdef np.ndarray[DTYPE_f, ndim=1] knots = np.array([ks[0]] * degree + ks + [ks[-1]] * degree, dtype=np.float64)
    cdef Py_ssize_t i, j, p
    cdef Py_ssize_t m = knots.shape[0]
    cdef np.ndarray[DTYPE_f, ndim=2] Bs = np.zeros( (degree+1, m), dtype=np.float64 )
    cdef Py_ssize_t n = Bs.shape[1]    
    cdef DTYPE_f term1, term2
    for j in range(m-1):
        if knots[j] <= u < knots[j+1]: 
            Bs[0, j] = 1.
    for p in range(1, degree+1):
        for i in range(n-p-1):
            term1 = 0.
            if knots[i+p] != knots[i]:
               term1 = (u - knots[i]) * Bs[p-1, i] / (knots[i+p] - knots[i])
            term2 = 0.
            if knots[i+p+1] != knots[i+1]:
                term2 = (knots[i+p+1] - u) * Bs[p-1, i+1] / (knots[i+p+1] - knots[i+1])
            Bs[p, i] = term1 + term2
    return Bs       


In [36]:
%%timeit
bspline_v4(knots, d, x)

The slowest run took 15.41 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 4.14 µs per loop


In [32]:
res4 = bspline_v4(knots, d, x)
diff = np.linalg.norm(res1 - res4)
print(diff)

0.0
